In [2]:
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
conn = sqlite3.connect('data.db')

In [4]:
sql_q = ('select * from from_nano;')
df = pd.read_sql(sql_q, conn)

In [5]:
df.datatime = pd.to_datetime(df.datatime)
df = df.set_index('datatime')

In [6]:
def clean_db(df):
    for i in tqdm(range(len(df.columns))):
        if i == 4:
            for j in range(len(df.iloc[:,i])):
                if not 100000 < float(df.iloc[j,i]) < 150000:
                    df.iloc[j,i] = df.iloc[j-1,i]
        else:
            for j in range(len(df.iloc[:,i])):
                if not 0 < float(df.iloc[j,i]) < 100:
                    df.iloc[j,i] = df.iloc[j-1,i]
    return df
                

In [7]:
df = clean_db(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 32.62it/s]


In [10]:
color1 = '#9467bd'
color2 = '#F08B00'

df2 = df[df.index > '2020-05-21 12:00:00']

trace1 = go.Scatter(
    x = df2.index,
    y = df2['air_temp1'],
    name='Air Temp',
    line = dict(
        color = color1
    )
)
trace2 = go.Scatter(
    x = df2.index,
    y = df2['soil_temp'] ,
    name='Soil Temp',
    yaxis='y3',
    line = dict(
        color = color2
    )
)

data = [trace1,trace2]
layout = go.Layout(
    title= "Tempreture",
    yaxis=dict(
        title='Air Temp (C)',
        titlefont=dict(
            color=color1
        ),
        tickfont=dict(
            color=color1
        )
    ),
    
#     yaxis2=dict(
#         title='Temp2',
#         overlaying='y',
#         side='right',
#         titlefont=dict(
#             color=color2
#         ),
#         tickfont=dict(
#             color=color2
#         )

#     ),
    
    yaxis3=dict(
        title='Soil Temp (C)',
        overlaying='y',
        side='right',
        titlefont=dict(
            color=color2
        ),
        tickfont=dict(
            color=color2
        )

    )

)
fig = go.Figure(data=data, layout=layout)
# fig.open()
# plot_url = py.iplot(fig)
# plot_url.show()

In [11]:
fig.write_image('fig.png')